# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [19]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 60

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [20]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [21]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)#输入和输出通道数分别为1和10
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)#输入和输出通道数分别为10和20
        self.conv2_drop = nn.Dropout2d()#随机选择输入的信道，将其设为0
        self.fc1 = nn.Linear(320, 50)#输入的向量大小和输出的大小分别为320和50
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))#conv->max_pool->relu
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))#conv->dropout->max_pool->relu
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))#fc->relu
        x = F.dropout(x, training=self.training)#dropout
        x = self.fc2(x)
        return F.log_softmax(x)

    
    def num_flat_features(self, x):
        size = x.size()[1:] # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

model = SimpleNet()
print(model)
# TODO:define loss function and optimiter
#criterion = nn.MSELoss()
#optimizer = torch.optim.RMSprop(model.parameters(), lr=0.01, alpha=0.9)
criterion = nn.NLLLoss2d()
optimizer = torch.optim.RMSprop(model.parameters(), lr = 0.001)

SimpleNet(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.5)
  (fc1): Linear(in_features=320, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=10, bias=True)
)


C:\Anaconda\lib\site-packages\torch\nn\modules\loss.py:206: UserWarning: NLLLoss2d has been deprecated. Please use NLLLoss instead as a drop-in replacement and see http://pytorch.org/docs/master/nn.html#torch.nn.NLLLoss for more details.
  warnings.warn("NLLLoss2d has been deprecated. "


Next, we can start to train and evaluate!

In [22]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    i = 0
    running_loss = 0
    print('EPOCHS', epoch)
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        #i = i + 1
        images, labels = Variable(images), Variable(labels)
        optimizer.zero_grad()
        output = model(images)
        #loss = F.nll_loss(output, labels)#负log似然损失
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        #running_loss += loss.data[0]
        #if i % 100 == 0:
         #   print(running_loss / 100)
         #   running_loss = 0;
        #print(loss)
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    correct = 0.0
    for images, labels in tqdm(test_loader):
        images, labels = Variable(images), Variable(labels)
        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)
        #predicted = torch.tensor(predicted)
        #pred_y = torch.max(output_pre_train, 1)[1].data.squeeze()
        correct += (predicted == labels).sum()
        #correct += output_pre_test.eq(labels).cpu().sum()
    test_accuracy = float(correct) / len(test_dataset.test_data)
    #print('test accuracy: ', accuracy)
    print('Testing accuracy: %0.2f%%' % (test_accuracy*100))
    
    correct = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = Variable(images), Variable(labels)
        outputs = model(images)
        _,predicted = torch.max(outputs.data, 1)
        #predicted = torch.tensor(predicted)
        #pred_y = torch.max(output_pre_train, 1)[1].data.squeeze()
        correct += (predicted == labels).sum()
        #correct += output_pre_test.eq(labels).cpu().sum()
    train_accuracy = float(correct) / len(train_dataset.train_data)
    #print('train accuracy: ', accuracy)
    print('Training accuracy: %0.2f%%' % (train_accuracy*100))


EPOCHS 0


100%|██████████| 78/78 [00:02<00:00, 34.24it/s]


Testing accuracy: 91.26%


100%|██████████| 468/468 [00:15<00:00, 31.09it/s]


Training accuracy: 91.01%
EPOCHS 1


...

Training accuracy: 97.36%
EPOCHS 59


100%|██████████| 78/78 [00:02<00:00, 31.68it/s]


Testing accuracy: 96.97%


100%|██████████| 468/468 [00:15<00:00, 30.91it/s]


Training accuracy: 97.30%


#### Q5:
Please print the training and testing accuracy.